In [1]:
import sys
sys.path.append("..")

import os
import kikuchipy as kp
from src import io, optimization
from pathlib import Path

In [2]:
optimized_correction_coefficients = {'tiltX_corr_angle': -0.5312103261068537,
                                     'tiltY_corr_angle': 1.2285403872457656, 
                                     'tiltZ_corr_angle': 0.06380671840706255, 
                                     'PCz': 3.890243796249578}

In [3]:
# Path to Si ECP reference pattern
si_ref_ecp_path = Path("../data/fcc_fe/")
si_ref_ecp_file = "01_Si_ref.tif"
si_ref_ecp_fname = os.path.join(si_ref_ecp_path, si_ref_ecp_file)

In [4]:
# Path to fcc_Fe master pattern generated by EMsoft
fe_MP_path = Path("../data/ebsd_master_pattern/")   
fe_MP_file = "Fe-master-20kV-1.h5"
fe_MP_fname = os.path.join(fe_MP_path, fe_MP_file)
fe_MP = kp.load(fe_MP_fname, projection="lambert", hemisphere="both", energy=20)

# Path to fcc_Fe .ctf file containing the EBSD euler angles
fe_xmap_path = Path("../data/ebsd_map/")  
fe_xmap_file = "20kv_26nA_15mm WD_4x4 bin_fcc_Fe Map.ctf"
fe_xmap_fname = os.path.join(fe_xmap_path, fe_xmap_file)
fe_xmap = io.load_xmap(fe_xmap_fname)

# Reference overview image 
sem_path = Path("../data/fcc_fe/")  
sem_file = "01_steel overview.tif"
sem_fname = os.path.join(sem_path, sem_file)

In [5]:
correlation = optimization.ipf_image_correlation(sem_fname, fe_xmap_fname)

Phase     Orientations         Name  Space group  Point group  Proper point group     Color
   -1      8358 (2.5%)  not_indexed         None         None                None         w
    1   322978 (97.5%)     Iron fcc        Fm-3m         m-3m                 432  tab:blue
Properties: bands, error, MAD, BC, BS
Scan unit: um


In [6]:
ipf_map = correlation.get_ipf_map(phase_name="Iron fcc", plot=False)
correlation.load_alignment_points(load_path="alignment_points.pkl")
ipf_warp_blended, _ = correlation.process(plot=False)

24 alignment points have been loaded.


In [7]:
%matplotlib qt
coord_results = correlation.interactive_blended_xmap(initial_coord=[ipf_warp_blended.shape[1]//2, ipf_warp_blended.shape[0]//2])

Euler angles at the image point: [99.38360000000002, 5.765999999999999, 81.1616]
Phase name at the image point: <name: Iron fcc. space group: Fm-3m. point group: m-3m. proper point group: 432. color: tab:blue>


Phase name at the image point: <name: Iron fcc. space group: Fm-3m. point group: m-3m. proper point group: 432. color: tab:blue>
Euler angles at the image point: [172.5426, 39.6595, 43.505100000000006]


In [8]:
tiltX_corr_angle_op = optimized_correction_coefficients["tiltX_corr_angle"] # positive direction => pattern moves DOWN
tiltY_corr_angle_op = optimized_correction_coefficients["tiltY_corr_angle"] # positive direction => pattern moves LEFT
tiltZ_corr_angle_op = optimized_correction_coefficients["tiltZ_corr_angle"] # positive direction => pattern rotates COUNTER-CLOCKWISE
PCz_op = optimized_correction_coefficients["PCz"] # Larger value => smaller angular range

correlation.interactive_rkp(ref_ECP_path= si_ref_ecp_fname,
                            RKP_masterpattern=fe_MP,
                            corr_angles=[tiltX_corr_angle_op, tiltY_corr_angle_op, tiltZ_corr_angle_op],
                            cam_length=PCz_op,
                            stage_mode="rot-tilt",
                            )

Finding bands that are in some pattern:
[########################################] | 100% Completed |  0.1s
Finding zone axes that are in some pattern:
[########################################] | 100% Completed |  0.1s
Calculating detector coordinates for bands and zone axes:
[########################################] | 100% Completed |  0.1s
